# LED-base Smoke Test

A tiny end-to-end sanity check for the `allenai/led-base-16384` model that the paper uses as the encoder–decoder baseline. It constructs a single inline example, loads the pretrained checkpoint, and generates one summary so you can confirm Transformers + CUDA are wired up (no dataset files required).

In [6]:
# @title Install dependencies
%pip install -q transformers==4.41.0 accelerate==0.28.0 sentencepiece

In [3]:
import os

# os.environ["HF_TOKEN"] = ""

In [ ]:
# @title Run LED-base on a single inline example
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

MODEL_ID = "allenai/led-base-16384"  # @param {"type": "string"}

example = {
    "text": (
        "Brief Hospital Course: Mr. Smith came in with fever, productive cough, and shortness of breath. "
        "Imaging was consistent with pneumonia and he required oxygen for two days. Antibiotics and inhalers were started, "
        "his breathing improved, and he was discharged home on day 4 with a plan for primary care follow-up."
    ),
    "summary": (
        "You were hospitalized for pneumonia that caused trouble breathing. We treated you with oxygen and antibiotics, "
        "you improved, and you can continue medication at home before seeing your primary doctor."
    ),
}

print("=== Reference summary ===")
print(example["summary"])
print("\n=== LED generation ===")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

inputs = tokenizer(
    example["text"],
    return_tensors="pt",
    truncation=True,
    max_length=2048,
)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
with torch.inference_mode():
    generation = model.generate(
        **inputs,
        max_new_tokens=256,
        num_beams=4,
    )
decoded = tokenizer.decode(generation[0], skip_special_tokens=True)
print(decoded)


=== Reference summary ===
You were hospitalized for pneumonia that caused trouble breathing. We treated you with oxygen and antibiotics, you improved, and you can continue medication at home before seeing your primary doctor.

=== LED generation ===
